# Open3D Guide: 11. Voxelization

Source: 

- Tutorial: [https://www.open3d.org/docs/latest/tutorial/Advanced/voxelization.html](https://www.open3d.org/docs/latest/tutorial/Advanced/voxelization.html).
- [`open3d.geometry.VoxelGrid`](https://www.open3d.org/docs/latest/python_api/open3d.geometry.VoxelGrid.html#open3d.geometry.VoxelGrid).
- [`open3d.geometry.Voxel`](https://www.open3d.org/docs/latest/python_api/open3d.geometry.Voxel.html#open3d.geometry.Voxel)

Summary of contents:

- A
- B

In [1]:
import sys
import os
import copy

# Add the directory containing 'examples' to the Python path
notebook_directory = os.getcwd()
parent_directory = os.path.dirname(notebook_directory)  # Parent directory
sys.path.append(parent_directory)

In [2]:
import open3d as o3d
import open3d.core as o3c
from examples import open3d_example as o3dex
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Voxelize from triangle mesh

In [3]:
# Get mesh
print('Input mesh')
mesh = o3dex.get_bunny_mesh()

# Optional: Fit to unit cube
#mesh.scale(
#    1.0 / np.max(mesh.get_max_bound() - mesh.get_min_bound()),
#    center=mesh.get_center()
#)

# Optional, for nice visualization
mesh.compute_vertex_normals()

# Visualize
o3d.visualization.draw_geometries([mesh])

Input mesh


In [4]:
# See all the properties of VoxelGrid
# https://www.open3d.org/docs/latest/python_api/open3d.geometry.VoxelGrid.html#open3d.geometry.VoxelGrid
print('Voxelization')
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(
    mesh,
    voxel_size=0.001
)
o3d.visualization.draw_geometries([voxel_grid])

Voxelization


## Voxels

In [5]:
# Get OCCUPIED voxels: get_voxels()
# Voxel: https://www.open3d.org/docs/latest/python_api/open3d.geometry.Voxel.html#open3d.geometry.Voxel
# Voxel properties: grid_index (3,), color (3,)
# NOTE: Voxels are not thought for setting values manually...
for i, voxel in enumerate(voxel_grid.get_voxels()):
    print("grid_index: ", voxel.grid_index) # an array if 3 ints
    print("color: ", voxel.color) # an array of  floats; we can modify colors to display better!
    if i > 10:
        break

grid_index:  [  4 106  80]
color:  [0. 0. 0.]
grid_index:  [110  17 107]
color:  [0. 0. 0.]
grid_index:  [72 93 80]
color:  [0. 0. 0.]
grid_index:  [ 64  77 100]
color:  [0. 0. 0.]
grid_index:  [129  41 101]
color:  [0. 0. 0.]
grid_index:  [  8 115  70]
color:  [0. 0. 0.]
grid_index:  [ 97  52 119]
color:  [0. 0. 0.]
grid_index:  [ 29 113  40]
color:  [0. 0. 0.]
grid_index:  [146  33  62]
color:  [0. 0. 0.]
grid_index:  [ 18 123  74]
color:  [0. 0. 0.]
grid_index:  [30  1 72]
color:  [0. 0. 0.]
grid_index:  [ 16 115 102]
color:  [0. 0. 0.]


## Voxel Cubes

In [6]:
# Get the voxel size and origin for computation
voxel_size = voxel_grid.voxel_size
origin = voxel_grid.origin

# Prepare to merge meshes
mesh = o3d.geometry.TriangleMesh()
for voxel in voxel_grid.get_voxels():
    # Calculate the center of the voxel
    voxel_center = origin + voxel.grid_index * voxel_size + voxel_size / 2.0
    # Create a cube mesh for each voxel
    cube = o3d.geometry.TriangleMesh.create_box(width=voxel_grid.voxel_size,
                                                height=voxel_grid.voxel_size,
                                                depth=voxel_grid.voxel_size)
    cube.translate(voxel_center - np.array([voxel_grid.voxel_size / 2] * 3))
    cube.paint_uniform_color([voxel_center[0] % 1, voxel_center[1] % 1, voxel_center[2] % 1])  # Set color
    mesh += cube

# Visualize the merged mesh
o3d.visualization.draw_geometries([mesh])

## Create a Voxelmap: A cartesian occupancy map

In [12]:
# Get the extent of the voxel grid and compute dimensions
min_bound = voxel_grid.get_min_bound()
max_bound = voxel_grid.get_max_bound()
voxel_size = voxel_grid.voxel_size
aabb = voxel_grid.get_axis_aligned_bounding_box()
print(f"voxel_grid.min_bound: {min_bound}")
print(f"voxel_grid.max_bound: {max_bound}")
print(f"voxel_grid.voxel_size: {voxel_size}")
print(f"AABB: {aabb}")

# Compute grid dimensions
Nx = int((max_bound[0] - min_bound[0]) / voxel_size)
Ny = int((max_bound[1] - min_bound[1]) / voxel_size)
Nz = int((max_bound[2] - min_bound[2]) / voxel_size)
print(f"Number of occupied voxels: {len(voxel_grid.get_voxels())}")
print(f"Number of total voxels: {Nx*Ny*Nz}")

# Create an empty numpy array for the voxel map
voxel_map = np.zeros((Nx, Ny, Nz), dtype=bool)

# Populate the voxel map
solid_voxel_count = 0
for voxel in voxel_grid.get_voxels():
    # NOTE: a voxel_index is a a tuple (3,) with ints
    # and we get a voxel_index for each occupied voxel!
    index = voxel.grid_index
    if 0 <= index[0] < Nx and 0 <= index[1] < Ny and 0 <= index[2] < Nz:
        voxel_map[index] = True
        solid_voxel_count += 1

# Output the shape of the voxel map
print("Voxel map shape:", voxel_map.shape)
print("Found solid voxels: ", solid_voxel_count)

voxel_grid.min_bound: [-0.0951899  0.0324874 -0.0623736]
voxel_grid.max_bound: [0.0618101 0.1874874 0.0596264]
voxel_grid.voxel_size: 0.001
AABB: AxisAlignedBoundingBox: min: (-0.0951899, 0.0324874, -0.0623736), max: (0.0618101, 0.187487, 0.0596264)
Number of occupied voxels: 82999
Number of total voxels: 2968870
Voxel map shape: (157, 155, 122)
Found solid voxels:  82999


In [16]:
# Now, we check that voxel_map is correct by plotting the voxel centers as points

# Create a point cloud
pcd = o3d.geometry.PointCloud()

# List to hold points and colors
points = []
colors = []

# Iterate through the voxel map
# Faster alternative, but without using voxel_map
# for voxel in voxel_grid.get_voxels():
#     point = voxel_grid.get_voxel_center_coordinate(voxel.grid_index)
#     points.append(point)
#     colors.append([1, 0, 0])  # Red color for occupied voxels
for i in range(Nx):
    for j in range(Ny):
        for k in range(Nz):
            if voxel_map[i, j, k]:  # Check if the voxel is occupied
                # Compute the center of the voxel
                # The manual computation with min_bound is WRONG?!
                # center_x = min_bound[0] + (i * voxel_size) + (voxel_size*0.5)
                # center_y = min_bound[1] + (j * voxel_size) + (voxel_size*0.5)
                # center_z = min_bound[2] + (k * voxel_size) + (voxel_size*0.5)
                (center_x, center_y, center_z) = voxel_grid.get_voxel_center_coordinate(np.asarray([i,j,k]))
                points.append([center_x, center_y, center_z])
                colors.append([1, 0, 0])
                
# Convert lists to Open3D vectors
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])